In [10]:
import sys 
import numpy as np

In [11]:
import re
#from Stemmer import Stemmer
from nltk.stem.snowball import SnowballStemmer
#from nltk.stem.porter import PorterStemmer

In [12]:
import sqlite3 as sq

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

In [14]:
def load_data(path):
    dbname = path
    data = { 'text':[],'tag':[] }
    conn = sq.connect(dbname)
    try:
        c = conn.cursor()
        for row in c.execute("SELECT * FROM data"):
            data['text'] += [row[1]]
            data['tag'] += [row[2]]
    finally:
        conn.close()
    return data

In [15]:
def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    text = re.sub( r'https?://[\S]+', ' url ', text) # замена интернет ссылок
    text = re.sub( r'[\w\./]+\.[a-z]+', ' url ', text) 
 
    text = re.sub( r'\d+[-/\.]\d+[-/\.]\d+', ' date ', text) # замена даты и времени
    text = re.sub( r'\d+ ?гг?', ' date ', text) 
    text = re.sub( r'\d+:\d+(:\d+)?', ' time ', text) 

    # text = re.sub( r'@\w+', ' tname ', text ) # замена имён twiter
    # text = re.sub( r'#\w+', ' htag ', text ) # замена хештегов

    text = re.sub( r'<[^>]*>', ' ', text) # удаление html тагов
    text = re.sub( r'[\W]+', ' ', text ) # удаление лишних символов

    stemmer = SnowballStemmer("russian")
    #text = ' '.join( stemmer.stem( text.split() ) ) 
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
           'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    remove = r'\b('+'|'.join(stw)+')\b'
    text = re.sub(remove,' ', text)
    
    text = re.sub( r'\b\w\b', ' ', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text

In [16]:
print("[i] загружаем данные...")
data = load_data('C:\\Users\\greg\\py_work\\texts\\rss-all.sqlite')
print("\tсчитано: ", len(data['text']))
print("\tколичество исходных категорий:",len(set(data['tag'])))

#stemmerS = SnowballStemmer("russian") 
#my_word = 'Аклиматизация'
#print(stemmerS.stem(my_word))   

print("[i] очистка данных...")
D = [ text_cleaner(t) for t in data['text'] ]
print("[i] готово.")

[i] загружаем данные...
	считано:  3196
	количество исходных категорий: 13
[i] очистка данных...
[i] готово.


In [17]:
print("[i] печать данных...")
print(D[:10])

[i] печать данных...
['  саудовск арав снял перв антитеррористическ мультфильм трехминутн лент вниман вниман отража точк зрен миров сообществ на войн развяза терроризм   поэт понят всем заяв режиссер короткометражк акр аг антитеррористическ мультфильм уж демонстрирова на кинофестивал   швейцар   герман арабск зрител смогут увидет ег на     ежегодн кинофестивал   дуба котор пройдет    digit  по  digit  декабр', 'вчер вечер   япон состоя премьер голливудск фильм     вызва негодован   связ   тем что эт девушк представл проститутк   игра их актрис китаянк мал тог что главн рол фильм мемуар   исполн китайск актрис он ещ   снят   калифорн   не   киот традицион мест обитан   американск режиссер по книг написа американц фильм показ котор происход   здан стадион для сум нос откровен сексуальн характер   внов оживля образ   как ночн бабочек однак   японск язык   означа искусн   девушк вид себ продолжательниц так традиц как чайн церемон икеба   поэз', 'российск кинорежиссер   генеральн директор к

In [18]:
n_clusters=13

print("[i] обучение кластеризатора...")
text_clstz = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ( 'km', KMeans(n_clusters=n_clusters)),
            #( 'km', KMeans(n_clusters=n_clusters, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0) )
            ])

text_clstz.fit(D)
data['tag'] = text_clstz.predict(D)

print("\tколичество кластеров:",len(set(data['tag'])))

[i] обучение кластеризатора...
	количество кластеров: 13


In [19]:
print((data['tag'])[:100])

[ 3  3  3  3  3  3  3  3  3  6  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3 12  3  3  3  3  9  3  3
  3  3  3  3  3  3  3  3  3  3  9  9  3  9  3  3  3  3  9  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3]
